In [1]:
# Parameters
TEST = True
QUICK_RUN = True


In [2]:
import pandas as pd
import numpy as np
import sys

In [3]:
sys.path.insert(0,"/usr/local/lib/python3.9/site-packages")

In [4]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import sessionmaker
import tweepy
import mysql.connector
from mysql.connector import Error
import json
from dateutil import parser
import time
import os
import subprocess
import pymysql
import requests
from bs4 import BeautifulSoup
import pprint

In [5]:
#streaming data from twitter and adding that data to AWS mysql

consumer_key = os.environ.get('consumer_key')
consumer_secret = os.environ.get('consumer_secret')
access_token = os.environ.get('access_token')
access_token_secret = os.environ.get('access_token_secret')
password = "password"
host_name = os.environ.get('host_name')
user = "group2"
aws_password = os.environ.get('aws_password')
db="group2db"
states = {
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AR': 'Arkansas',
    'AS': 'American Samoa',
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'GU': 'Guam',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MP': 'Northern Mariana Islands',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NA': 'National',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'PR': 'Puerto Rico',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VI': 'Virgin Islands',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming'
}

def connect(username, created_at, tweet, retweet_count, place, location):

    """
    connect to MySQL database and insert twitter data
    """
    global cursor
    try:
        con = mysql.connector.connect(host=host_name,
                                      database=db, user=user, password=aws_password, charset='utf8')
        if con.is_connected():
            """
            Insert twitter data
            """
            cursor = con.cursor()
            #######creating table is AWS mysql
#             create_table="""CREATE TABLE tweets_with_location (tweet_id INT(11) not null auto_increment primary key, username VARCHAR(255), 
#                created_at VARCHAR(45), tweet TEXT,
#                retweet_count INT(11), location VARCHAR(100), place VARCHAR(100));"""
#             cursor.execute(create_table)
            # twitter
            
            if location in states.values()or location in states.keys():
                if location in states.keys():
                    location = states.get(location)
                query = "INSERT INTO tweets_with_location (username, created_at, tweet, retweet_count,place, location) VALUES (%s, %s, %s, %s, %s, %s)"
                cursor.execute(query, (username, created_at, tweet, retweet_count, place, location))
                con.commit()

    except Error as e:
        print(e)
    cursor.close()
    con.close()
    return
# Tweepy class to access Twitter API
class Streamlistener(tweepy.StreamListener):
    max_tweets = 20
    tweet_count = 0
    def on_connect(self):
        print("You are connected to the Twitter API")
    def on_error(self):
        if status_code != 200:
            print("error found")
            # returning false disconnects the stream
            return False
    """
    This method reads in tweet data as Json
    and extracts the data we want.
    """
    def on_data(self, data):
        try:
            self.tweet_count+=1
            if(self.tweet_count==self.max_tweets):
                print("completed")
                return(False)
            else:
                raw_data = json.loads(data)
                if 'text' in raw_data :

                    username = raw_data['user']['screen_name']
                    created_at = parser.parse(raw_data['created_at'])
                    tweet = raw_data['text']
                    retweet_count = raw_data['retweet_count']
                    if raw_data['place'] is not None:
                        place = raw_data['place']['country']
                    else:
                        place = None
                        location = raw_data['user']['location']
                        # insert data just collected into MySQL database
                        connect(username, created_at, tweet, retweet_count, place, location)
        #                     print("Tweet colleted at: {} ".format(str(created_at)))
                            
        except Error as e:
            print('completed')
            print(e)
if __name__ == '__main__':
    # # #Allow user input
   
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True)
    # create instance of Streamlistener
    listener = Streamlistener(api=api)
    stream = tweepy.Stream(auth, listener=listener)
    track = ['covid vaccine', 'covid-19 vaccine', 'Covid-19 vaccine','Covid 19 vaccine']
    # choose what we want to filter by
    stream.filter(track=track, languages=['en'])

You are connected to the Twitter API


completed
